In [ ]:
import pandas as pd

# Load your dataset
data_path = '/content/44.csv'  # Update this to your dataset's file path
data = pd.read_csv(data_path)

# Assuming your dataset's date column might need conversion to datetime and extraction of the year
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%y')  # Adjust the format as per your dataset
data['year'] = data['date'].dt.year

# Ensuring the dataset is within the 2001-2023 range
data = data[(data['year'] >= 2001) & (data['year'] <= 2023)]

# Define the list of predictions to correct
predictions_corrected = ['sc-1W', 'sc-1L', '1-Ws', '1-Ls']

# Define the function to calculate metrics
def calculate_metrics_with_accuracy(df, true_label, predictions, true_label_sc=None):
    metrics = {}
    for pred in predictions:
        if pred.startswith('sc'):
            # For 'sc' strategies, use "True Label For sc"
            tp = ((df[true_label_sc] == 'SPY') & (df[pred] == 'SPY')).sum()
            tn = ((df[true_label_sc] == 'Cash') & (df[pred] == 'Cash')).sum()
            fp = ((df[true_label_sc] == 'Cash') & (df[pred] == 'SPY')).sum()
            fn = ((df[true_label_sc] == 'SPY') & (df[pred] == 'Cash')).sum()
        else:
            # Original calculation for other strategies
            tp = ((df[true_label] == 'Growth') & (df[pred] == 'Growth')).sum()
            tn = ((df[true_label] == 'Value') & (df[pred] == 'Value')).sum()
            fp = ((df[true_label] == 'Value') & (df[pred] == 'Growth')).sum()
            fn = ((df[true_label] == 'Growth') & (df[pred] == 'Value')).sum()
        accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        pp = tp + fp
        pn = tn + fn
        metrics[pred.replace(' ', '_').replace('-', '_')] = {
            'TPR': tpr, 'TNR': tnr, 'Accuracy': accuracy,
            'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn
            , 'PP': pp, 'PN': pn
        }
    return metrics

# Calculate metrics including accuracy for each year within the specified range
metrics_by_year_with_accuracy = {}
for year, group in data.groupby('year'):
    metrics_by_year_with_accuracy[year] = calculate_metrics_with_accuracy(group, 'True Label', predictions_corrected, true_label_sc='True Label For sc')

# Reformatting the output for the specified order and metric display
output_data = []
for year, metrics in metrics_by_year_with_accuracy.items():
    row = [year]
    for prediction in predictions_corrected:
        pred_key = prediction.replace(' ', '_').replace('-', '_')
        row.extend([
            metrics[pred_key]['TPR'], metrics[pred_key]['TNR'], metrics[pred_key]['Accuracy']
        ])
    output_data.append(row)

# Creating a DataFrame for the restructured data
columns = ['Year']
for pred in predictions_corrected:
    pred_key = pred.replace(' ', '_').replace('-', '_')
    columns.extend([f'{pred_key}_TPR', f'{pred_key}_TNR', f'{pred_key}_Accuracy'])

restructured_df = pd.DataFrame(output_data, columns=columns)
print(restructured_df)

# 保存到CSV文件
csv_file_path = '/content/metrics_by_year_restructured_df.csv'  # 更改为您希望保存的路径
restructured_df.to_csv(csv_file_path, index=False)

    Year  sc_1W_TPR  sc_1W_TNR  sc_1W_Accuracy  sc_1L_TPR  sc_1L_TNR  \
0   2001   0.488000   0.479675        0.483871   0.512000   0.520325   
1   2002   0.391667   0.454545        0.424603   0.608333   0.545455   
2   2003   0.541667   0.388889        0.476190   0.458333   0.611111   
3   2004   0.586207   0.429907        0.519841   0.413793   0.570093   
4   2005   0.535714   0.419643        0.484127   0.464286   0.580357   
5   2006   0.588652   0.472727        0.537849   0.411348   0.527273   
6   2007   0.489209   0.366071        0.434263   0.510791   0.633929   
7   2008   0.500000   0.511811        0.505929   0.500000   0.488189   
8   2009   0.560284   0.432432        0.503968   0.439716   0.567568   
9   2010   0.571429   0.409524        0.503968   0.428571   0.590476   
10  2011   0.566176   0.482759        0.527778   0.433824   0.517241   
11  2012   0.589928   0.495495        0.548000   0.410072   0.504505   
12  2013   0.570470   0.378641        0.492063   0.429530   0.62

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
# Load your dataset
data_path = '/content/44.csv'  # Update this to your dataset's file path
data = pd.read_csv(data_path)
# 假设您的日期列需要转换
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%y')  # 根据您的日期格式调整
data['year'] = data['date'].dt.year

# 确保数据集在2001-2023年范围内
data = data[(data['year'] >= 2001) & (data['year'] <= 2023)]

# 定义预测列名列表
predictions_corrected = ['sc-1W', 'sc-1L', '1-Ws', '1-Ls']

# 定义计算指标和AUC的函数
def calculate_metrics_with_accuracy_and_auc(df, true_label, predictions, true_label_sc=None):
    metrics = {}
    for pred in predictions:
        pred_key = pred.replace(' ', '_').replace('-', '_')
        metrics[pred_key] = {'TPR': 0, 'TNR': 0, 'Accuracy': 0, 'AUC': 0}

        # 初始化真实值和预测概率列表
        true_binary = []
        pred_probabilities = []

        if pred.startswith('sc'):
            for index, row in df.iterrows():
                true_binary.append(1 if row[true_label_sc] == 'SPY' else 0)
                pred_probabilities.append(1 if row[pred] == 'SPY' else 0)
        else:
            for index, row in df.iterrows():
                true_binary.append(1 if row[true_label] == 'Growth' else 0)
                pred_probabilities.append(1 if row[pred] == 'Growth' else 0)

        # 计算AUC
        metrics[pred_key]['AUC'] = roc_auc_score(true_binary, pred_probabilities)
        # 省略的准确度、TPR、TNR计算请在此处添加

    return metrics

# 分年度计算指标
metrics_by_year_with_auc = {}
for year, group in data.groupby('year'):
    metrics_by_year_with_auc[year] = calculate_metrics_with_accuracy_and_auc(group, 'True Label', predictions_corrected, true_label_sc='True Label For sc')

# 准备数据以保存到CSV
output_data = []
for year, metrics in metrics_by_year_with_auc.items():
    row = [year]
    for pred in predictions_corrected:
        pred_key = pred.replace(' ', '_').replace('-', '_')
        row.extend([
            metrics[pred_key]['TPR'], metrics[pred_key]['TNR'], metrics[pred_key]['Accuracy'], metrics[pred_key]['AUC']
        ])
    output_data.append(row)

# 创建DataFrame
columns = ['Year']
for pred in predictions_corrected:
    pred_key = pred.replace(' ', '_').replace('-', '_')
    columns.extend([f'{pred_key}_TPR', f'{pred_key}_TNR', f'{pred_key}_Accuracy', f'{pred_key}_AUC'])

metrics_df = pd.DataFrame(output_data, columns=columns)

# 保存到CSV文件
csv_file_path = '/content/metrics_by_year_with_auc.csv'  # 更改为您希望保存的路径
metrics_df.to_csv(csv_file_path, index=False)

print(f'Metrics saved to {csv_file_path}')


Metrics saved to /content/metrics_by_year_with_auc.csv


In [ ]:
import pandas as pd
data_path = '/content/44.csv'  # Update this to your dataset's file path
data_new = pd.read_csv(data_path)

NameError: name 'pd' is not defined

In [ ]:
# Convert the 'date' column to datetime and extract the year to a new 'year' column
data_new['date'] = pd.to_datetime(data_new['date'], format='%m/%d/%y')
data_new['year'] = data_new['date'].dt.year

# Adjusted aggregated data structure to include SPY and Cash counts
aggregated_data = {
    'Year': [],
    'Value': [],
    'Growth': [],
    'SPY': [],
    'Cash': [],
    'R(Value)': [],
    'R(Growth)': [],
    'R(SPY)': [],
    'R(Cash)': []
}

# Aggregate the data
for year, group in data_new.groupby('year'):
    # For Value
    value_data = group[group['True Label'] == 'Value']
    value_counts = value_data.shape[0]
    value_sum = value_data['spyv'].sum()
    average_r_value = value_sum / value_counts if value_counts else 0

    # For Growth
    growth_data = group[group['True Label'] == 'Growth']
    growth_counts = growth_data.shape[0]
    growth_sum = growth_data['spyg'].sum()
    average_r_growth = growth_sum / growth_counts if growth_counts else 0

    # For SPY (S+)
    spy_data = group[group['True Label For sc'] == 'SPY']
    spy_counts = spy_data.shape[0]
    spy_sum = spy_data['spy'].sum()
    average_s_plus = spy_sum / spy_counts if spy_counts else 0

    # For Cash (S-)
    cash_data = group[group['True Label For sc'] == 'Cash']
    cash_counts = cash_data.shape[0]
    cash_sum = cash_data['spy'].sum()
    average_s_minus = cash_sum / cash_counts if cash_counts else 0

    # Append to the dictionary
    aggregated_data['Year'].append(year)
    aggregated_data['Value'].append(value_counts)
    aggregated_data['Growth'].append(growth_counts)
    aggregated_data['SPY'].append(spy_counts)
    aggregated_data['Cash'].append(cash_counts)
    aggregated_data['R(Value)'].append(average_r_value)
    aggregated_data['R(Growth)'].append(average_r_growth)
    aggregated_data['R(SPY)'].append(average_s_plus)
    aggregated_data['R(Cash)'].append(average_s_minus)

# Convert the dictionary to a DataFrame
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df


,Year,Value,Growth,SPY,Cash,R(Value),R(Growth),R(SPY),R(Cash)
0,2001,127,121,125,123,-0.000375,0.015424,0.010119,-0.011106
1,2002,134,118,120,132,-0.000725,0.011147,0.012598,-0.013031
2,2003,125,127,144,108,0.002781,0.005022,0.007969,-0.008199
3,2004,132,120,145,107,0.001948,0.002601,0.005153,-0.005975
4,2005,131,121,140,112,0.001099,0.002448,0.004819,-0.005555
5,2006,142,109,141,110,0.000983,0.002936,0.004719,-0.004666
6,2007,113,138,139,112,0.004105,0.000024,0.006726,-0.007787
7,2008,128,125,126,127,0.007092,-0.003099,0.015555,-0.018374
8,2009,107,145,141,111,0.005848,0.001942,0.011709,-0.012448
9,2010,127,125,147,105,0.001158,0.003538,0.007314,-0.008751


In [ ]:
import pandas as pd
data_path = '/content/44.csv'  # Update this to your dataset's file path
data_new = pd.read_csv(data_path)

import pandas as pd

# Assuming data_new is your DataFrame and it's already available with necessary columns.

# Convert the 'date' column to datetime and extract the year to a new 'year' column
data_new['date'] = pd.to_datetime(data_new['date'], format='%m/%d/%y')
data_new['year'] = data_new['date'].dt.year

# Adjusted aggregated data structure to include SPY, Cash counts, and new R-G, R+V columns
aggregated_data = {
    'Year': [],
    'Value': [],
    'Growth': [],
    'SPY': [],
    'Cash': [],
    'R(Value)': [],
    'R(Growth)': [],
    'R(SPY)': [],
    'R(Cash)': [],
    'R-G': [],
    'R+V': []
}

# Aggregate the data
for year, group in data_new.groupby('year'):
    # For Value
    value_data = group[group['True Label'] == 'Value']
    value_counts = value_data.shape[0]
    value_sum = value_data['spyv'].sum()
    average_r_value = value_sum / value_counts if value_counts else 0

    # Compute R-G (average 'growth_return' when true label is 'Value')
    growth_return_sum = value_data['spyg'].sum()
    average_r_growth_for_value = growth_return_sum / value_counts if value_counts else 0

    # For Growth
    growth_data = group[group['True Label'] == 'Growth']
    growth_counts = growth_data.shape[0]
    growth_sum = growth_data['spyg'].sum()
    average_r_growth = growth_sum / growth_counts if growth_counts else 0

    # Compute R+V (average 'value_return' when true label is 'Growth')
    value_return_sum = growth_data['spyv'].sum()
    average_r_value_for_growth = value_return_sum / growth_counts if growth_counts else 0

    # For SPY (S+)
    spy_data = group[group['True Label For sc'] == 'SPY']
    spy_counts = spy_data.shape[0]
    spy_sum = spy_data['spy'].sum()
    average_s_plus = spy_sum / spy_counts if spy_counts else 0

    # For Cash (S-)
    cash_data = group[group['True Label For sc'] == 'Cash']
    cash_counts = cash_data.shape[0]
    cash_sum = cash_data['spy'].sum()
    average_s_minus = cash_sum / cash_counts if cash_counts else 0

    # Append to the dictionary
    aggregated_data['Year'].append(year)
    aggregated_data['Value'].append(value_counts)
    aggregated_data['Growth'].append(growth_counts)
    aggregated_data['SPY'].append(spy_counts)
    aggregated_data['Cash'].append(cash_counts)
    aggregated_data['R(Value)'].append(average_r_value)
    aggregated_data['R(Growth)'].append(average_r_growth)
    aggregated_data['R(SPY)'].append(average_s_plus)
    aggregated_data['R(Cash)'].append(average_s_minus)
    aggregated_data['R-G'].append(average_r_growth_for_value)
    aggregated_data['R+V'].append(average_r_value_for_growth)

# Convert the dictionary to a DataFrame
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df


,Year,Value,Growth,SPY,Cash,R(Value),R(Growth),R(SPY),R(Cash),R-G,R+V
0,2001,127,121,125,123,-0.000375,0.015424,0.010119,-0.011106,-0.016406,0.000038
1,2002,134,118,120,132,-0.000725,0.011147,0.012598,-0.013031,-0.012301,-0.000601
2,2003,125,127,144,108,0.002781,0.005022,0.007969,-0.008199,-0.002973,-0.000858
3,2004,132,120,145,107,0.001948,0.002601,0.005153,-0.005975,-0.001929,-0.001063
4,2005,131,121,140,112,0.001099,0.002448,0.004819,-0.005555,-0.002012,-0.000719
5,2006,142,109,141,110,0.000983,0.002936,0.004719,-0.004666,-0.001606,0.000556
6,2007,113,138,139,112,0.004105,0.000024,0.006726,-0.007787,0.000968,-0.003160
7,2008,128,125,126,127,0.007092,-0.003099,0.015555,-0.018374,-0.000087,-0.010192
8,2009,107,145,141,111,0.005848,0.001942,0.011709,-0.012448,0.000615,-0.002962
9,2010,127,125,147,105,0.001158,0.003538,0.007314,-0.008751,-0.002157,0.000102


from matplotlib import pyplot as plt
aggregated_df['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['Value'].plot(kind='hist', bins=20, title='Value')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['Growth'].plot(kind='hist', bins=20, title='Growth')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['SPY'].plot(kind='hist', bins=20, title='SPY')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Year']
  ys = series['Cash']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = aggregated_df.sort_values('Year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('Cash')

from matplotlib import pyplot as plt
aggregated_df['Year'].plot(kind='line', figsize=(8, 4), title='Year')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['Value'].plot(kind='line', figsize=(8, 4), title='Value')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['Growth'].plot(kind='line', figsize=(8, 4), title='Growth')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
aggregated_df['SPY'].plot(kind='line', figsize=(8, 4), title='SPY')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
csv_file_path = '/content/metrics_by_year_with_auc.csv'  # 更改为您希望保存的路径
metrics_df.to_csv(csv_file_path, index=False)

print(f'Metrics saved to {csv_file_path}')


In [ ]:
import pandas as pd

# Load the data
data = pd.read_csv('/content/79.csv')

# Calculate tp, tn, fp, fn
data['tp'] = data['actual postive'] * data['TPR']
data['tn'] = data['actual negative'] * data['TNR']
data['fp'] = data['actual negative'] - data['tn']
data['fn'] = data['actual postive'] - data['tp']

# Calculate pp and pn
data['pp'] = data['tp'] + data['fp']
data['pn'] = data['tn'] + data['fn']

# Show the result with new calculated columns
data[['Year', 'pp', 'pn', 'tp', 'fp', 'tn', 'fn']]


,Year,pp,pn,tp,fp,tn,fn
0,2001,123.0,125.0,64.0,59.0,64.0,61.0
1,2002,133.0,119.0,73.0,60.0,72.0,47.0
2,2003,108.0,144.0,66.0,42.0,66.0,78.0
3,2004,106.0,146.0,60.0,46.0,61.0,85.0
4,2005,112.0,140.0,65.0,47.0,65.0,75.0
5,2006,110.0,141.0,58.0,52.0,58.0,83.0
6,2007,112.0,139.0,71.0,41.0,71.0,68.0
7,2008,128.0,125.0,63.0,65.0,62.0,63.0
8,2009,110.0,142.0,62.0,48.0,63.0,79.0
9,2010,106.0,146.0,63.0,43.0,62.0,84.0


In [ ]:
data.columns

Index(['date', 'spy', 'spyg', 'spyv', 'True Label', 'True Label For sc',
       '1 ws', '1 ld ', 'sc 1-ws', 'sc 1-ls', 'year'],
      dtype='object')

In [ ]:
import pandas as pd

# Assuming this is how the data is loaded and structured
data_path = '/content/82.csv'  # User needs to update this path
data = pd.read_csv(data_path)
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data = data[(data['year'] >= 2001) & (data['year'] <= 2023)]

# List of predictions
predictions_corrected = ['sc 1-ls', '1 ld ']

# Function to calculate the metrics
def calculate_metrics_with_counts(df, true_label, predictions, true_label_sc=None):
    metrics = {}
    for pred in predictions:
        if pred.startswith('sc'):
            tp = ((df[true_label_sc] == 'SPY') & (df[pred] == 'SPY')).sum()
            tn = ((df[true_label_sc] == 'Cash') & (df[pred] == 'Cash')).sum()
            fp = ((df[true_label_sc] == 'Cash') & (df[pred] == 'SPY')).sum()
            fn = ((df[true_label_sc] == 'SPY') & (df[pred] == 'Cash')).sum()
        else:
            tp = ((df[true_label] == 'Growth') & (df[pred] == 'Growth')).sum()
            tn = ((df[true_label] == 'Value') & (df[pred] == 'Value')).sum()
            fp = ((df[true_label] == 'Value') & (df[pred] == 'Growth')).sum()
            fn = ((df[true_label] == 'Growth') & (df[pred] == 'Value')).sum()

        pp = tp + fp
        pn = tn + fn
        prevalence = (tp + fn) / (tp + fp + tn + fn)
        ppv = tp / pp if pp > 0 else 0
        npv = tn / pn if pn > 0 else 0
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        metrics[pred.replace(' ', '_').replace('-', '_')] = {
            'TP': tp, 'FP': fp, 'TN': tn, 'FN': fn, 'PP': pp, 'PN': pn,
            'Prevalence': prevalence, 'PPV': ppv, 'NPV': npv,
            'TPR': tpr, 'TNR': tnr
        }
    return metrics

# Calculate metrics for each year
metrics_by_year = {}
for year, group in data.groupby('year'):
    metrics_by_year[year] = calculate_metrics_with_counts(group, 'True Label', predictions_corrected, true_label_sc='True Label For sc')

# Prepare output data
output_data = []
for year, metrics in metrics_by_year.items():
    row = [year]
    for prediction in predictions_corrected:
        pred_key = prediction.replace(' ', '_').replace('-', '_')
        row.extend([
            metrics[pred_key]['TP'], metrics[pred_key]['FP'], metrics[pred_key]['TN'], metrics[pred_key]['FN'],
            metrics[pred_key]['PP'], metrics[pred_key]['PN'],
            metrics[pred_key]['Prevalence'], metrics[pred_key]['PPV'], metrics[pred_key]['NPV'],
            metrics[pred_key]['TPR'], metrics[pred_key]['TNR']
        ])
    output_data.append(row)

# Define columns for the DataFrame
columns = ['Year']
for pred in predictions_corrected:
    pred_key = pred.replace(' ', '_').replace('-', '_')
    columns.extend([
        f'{pred_key}_TP', f'{pred_key}_FP', f'{pred_key}_TN', f'{pred_key}_FN',
        f'{pred_key}_PP', f'{pred_key}_PN',
        f'{pred_key}_Prevalence', f'{pred_key}_PPV', f'{pred_key}_NPV',
        f'{pred_key}_TPR', f'{pred_key}_TNR'
    ])

# Create DataFrame for output
restructured_df = pd.DataFrame(output_data, columns=columns)
print(restructured_df)

# Save to CSV file
csv_file_path = '/content/metrics_by_year_with_counts_and_rates.csv'  # Update path as needed
restructured_df.to_csv(csv_file_path, index=False)


    Year  sc_1_ls_TP  sc_1_ls_FP  sc_1_ls_TN  sc_1_ls_FN  sc_1_ls_PP  \
0   2001          64          59          64          61         123   
1   2002          73          60          72          47         133   
2   2003          66          42          66          78         108   
3   2004          60          46          61          85         106   
4   2005          65          47          65          75         112   
5   2006          58          52          58          83         110   
6   2007          71          41          71          68         112   
7   2008          63          65          62          63         128   
8   2009          62          48          63          79         110   
9   2010          63          43          62          84         106   
10  2011          59          56          60          77         115   
11  2012          57          55          56          82         112   
12  2013          64          39          64          85        

In [ ]:
import pandas as pd

# Load your dataset
data_path = '/content/44.csv'  # Update this to your dataset's file path
data = pd.read_csv(data_path)

# Assuming your dataset's date column might need conversion to datetime and extraction of the year
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%y')  # Adjust the format as per your dataset
data['year'] = data['date'].dt.year

# Ensuring the dataset is within the 2001-2023 range
data = data[(data['year'] >= 2001) & (data['year'] <= 2023)]

# Define the list of predictions to correct
predictions_corrected = ['sc-1W', 'sc-1L', '1-Ws', '1-Ls']

# Define the function to calculate metrics
def calculate_metrics_with_accuracy(df, true_label, predictions, true_label_sc=None):
    metrics = {}
    for pred in predictions:
        if pred.startswith('sc'):
            # For 'sc' strategies, use "True Label For sc"
            tp = ((df[true_label_sc] == 'SPY') & (df[pred] == 'SPY')).sum()
            tn = ((df[true_label_sc] == 'Cash') & (df[pred] == 'Cash')).sum()
            fp = ((df[true_label_sc] == 'Cash') & (df[pred] == 'SPY')).sum()
            fn = ((df[true_label_sc] == 'SPY') & (df[pred] == 'Cash')).sum()
        else:
            # Original calculation for other strategies
            tp = ((df[true_label] == 'Growth') & (df[pred] == 'Growth')).sum()
            tn = ((df[true_label] == 'Value') & (df[pred] == 'Value')).sum()
            fp = ((df[true_label] == 'Value') & (df[pred] == 'Growth')).sum()
            fn = ((df[true_label] == 'Growth') & (df[pred] == 'Value')).sum()
        accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        metrics[pred.replace(' ', '_').replace('-', '_')] = {
            'TPR': tpr, 'TNR': tnr, 'Accuracy': accuracy,
            'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn
        }
    return metrics

# Calculate metrics including accuracy for each year within the specified range
metrics_by_year_with_accuracy = {}
actual_labels_count = {}
for year, group in data.groupby('year'):
    metrics_by_year_with_accuracy[year] = calculate_metrics_with_accuracy(group, 'True Label', predictions_corrected, true_label_sc='True Label For sc')
    # Count actual labels for each year
    actual_labels_count[year] = group['True Label'].value_counts().to_dict()

# Reformatting the output for the specified order and metric display
output_data = []
for year, metrics in metrics_by_year_with_accuracy.items():
    row = [year]
    for prediction in predictions_corrected:
        pred_key = prediction.replace(' ', '_').replace('-', '_')
        row.extend([
            metrics[pred_key]['TPR'], metrics[pred_key]['TNR'], metrics[pred_key]['Accuracy'],
            metrics[pred_key]['TP'], metrics[pred_key]['TN'], metrics[pred_key]['FP'], metrics[pred_key]['FN']
        ])
    # Append actual label counts for each year
    label_counts = actual_labels_count.get(year, {})
    for label, count in label_counts.items():
        row.append(f"{label}: {count}")
    output_data.append(row)

# Creating a DataFrame for the restructured data
columns = ['Year']
for pred in predictions_corrected:
    pred_key = pred.replace(' ', '_').replace('-', '_')
    columns.extend([
        f'{pred_key}_TPR', f'{pred_key}_TNR', f'{pred_key}_Accuracy',
        f'{pred_key}_TP', f'{pred_key}_TN', f'{pred_key}_FP', f'{pred_key}_FN'
    ])
# Extend columns with labels
columns.extend(['Label Counts Postive'])
columns.extend(['Label Counts Negetive'])
restructured_df = pd.DataFrame(output_data, columns=columns)
print(restructured_df)

# Save to CSV file
csv_file_path = '/content/metrics_by_year_restructured_df.csv'  # Update the path as needed
restructured_df.to_csv(csv_file_path, index=False)

    Year  sc_1W_TPR  sc_1W_TNR  sc_1W_Accuracy  sc_1W_TP  sc_1W_TN  sc_1W_FP  \
0   2001   0.488000   0.479675        0.483871        61        59        64   
1   2002   0.391667   0.454545        0.424603        47        60        72   
2   2003   0.541667   0.388889        0.476190        78        42        66   
3   2004   0.586207   0.429907        0.519841        85        46        61   
4   2005   0.535714   0.419643        0.484127        75        47        65   
5   2006   0.588652   0.472727        0.537849        83        52        58   
6   2007   0.489209   0.366071        0.434263        68        41        71   
7   2008   0.500000   0.511811        0.505929        63        65        62   
8   2009   0.560284   0.432432        0.503968        79        48        63   
9   2010   0.571429   0.409524        0.503968        84        43        62   
10  2011   0.566176   0.482759        0.527778        77        56        60   
11  2012   0.589928   0.495495        0.

In [ ]:
import pandas as pd

# Load the data from the uploaded file
file_path = '/content/84.csv'  # Ensure the correct file path is used
data = pd.read_csv(file_path)

# Do not remove the first day; copy the whole data instead
new_data_filtered = data.copy()

# Assign the return based on the predicted label $P_{i}$
new_data_filtered["$r_{i}$"] = new_data_filtered.apply(
    lambda row: row['growth'] if row['$P_{i}$'] == '+' else row['value'], axis=1
)

# Initialize B_i for the first day in the filtered data to 100 (assumed for d0)
new_data_filtered.loc[0, "$B_{i}$"] = 100  # Assuming the first row (d0) starts with B_i = 100

# Iteratively update B_i for the rest of the days, starting from the first day (d1)
for i in range(1, len(new_data_filtered)):
    new_data_filtered.loc[i, "$B_{i}$"] = new_data_filtered.loc[i - 1, "$B_{i}$"] + new_data_filtered.loc[i, "$r_{i}$"]

# Display the updated dataframe
print(new_data_filtered)# Save the DataFrame to a CSV file
output_file_path = '/content/updated_data.csv'  # Specify the desired output file path
new_data_filtered.to_csv(output_file_path, index=False)  # Set index=False to not save row indices


         day  growth  value  $r_{i}$  $B_{i}$ $T_{i}$ $P_{i}$
0    $d_{0}$       3      1        1      100       +     $*$
1    $d_{1}$       2      0        2      102       +       +
2    $d_{2}$      -3     -2       -3       99       -       +
3    $d_{3}$      -2     -1       -1       98       -       -
4    $d_{4}$       2      1        1       99       +       -
5    $d_{5}$       3      2        3      102       +       +
6    $d_{6}$       0      1        0      102       -       +
7    $d_{7}$      -1     -2       -2      100       +       -
8    $d_{8}$      -3      1       -3       97       -       +
9    $d_{9}$       3      2        2       99       +       -
10  $d_{10}$      -2     -1       -2       97       -       +
